# AOC 2022

In [220]:
\cd notebooks

'io



# Advent of Code 2022 - 07

URL: https://adventofcode.com/2022/day/7

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/07.k):

In [173]:
x:0:"i/07"
s:,0 /stack: current sizes of ancestors of current directory
t:!0 /total sizes of already visited directories
/ {$["$ cd .."~x;[t,:-1#s;s::-1_s]
/   "$ cd /" ~x;[t,: 1_s;s:: 1#s]
/   "$ cd "~5#x;s,:0
/   ~"0:"'*x;s+:.*" "\x]}'x;

{$["$ cd .."~x;[t,:-1#s;s::-1_s] ; "$ cd /" ~x;[t,: 1_s;s:: 1#s];"$ cd "~5#x;s,:0;~"0:"'*x;s+:.*" "\x]}'x;

t,:s
+/(1e5>)#t       /part1
&/((-4e7+*s)>)_t /part2

1642503
6999588


The same as in day2, we encounter the issue in jupyter notebooks where multiline statements are not correctly parsed.

In this case, there's our first conditional statement ever, that ngn writes in a multiline fashion, but we can't experiment in the repl.

This challenge revolves around accumulating sizes of partial paths, with simulated files and directories.

First the easy 3 lines: 

- Read input file
- `s` is a list with `0`
- `t` is `!0`

In [7]:
x:0:"i/07"
s:,0 /stack: current sizes of ancestors of current directory
t:!0 /total sizes of already visited directories

Worth mentioning that the solution here relies on one fact about the data, that is not explicit in the challenge, but it's true in all inputs: The directories are only visited once. There won't be `ls`s repeatedly thrown to the same directories.

That makes this solution viable, as it blindly pushes results to `t` on each `cd ..` or `cd /`.

Now the conditional. Conditionals in k behave like Lisp's `cond`: There's an indefinite number of pairs of expressions `$[e1;e2;e3;e4;....e2n]`.

`e1` is evaluated, and if true, `e2` is evaluated and the conditional returns the result of `e2` as its result. Else, `e3` is evaluated, and if true `e4` is returned...

If the number of expressions is odd, the last one works like a final `else`.

Foreach line `'x`, run through the lambda that contains a single cond:
If the line looks like a "cd ..", evaluate `[t,:-1#s;s::-1_s]`. See that we can evaluate multiple expressions as one by putting them in square brackets.
`[t,:-1#s;s::-1_s]` is 2 statements: `t,:-1#s` ammends t to copy the last element of `s` to be the last element of t, and then, ammend s to remove that last element.


In [17]:
t:,0
s:1 2 3
:t,:-1#s
s
:s:-1_s

0 3
1 2 3
1 2


The second condition, `"$ cd /" ~x;[t,: 1_s;s:: 1#s]`.
If the line is "cd /", then push all but the first element of s to the end of t

In [19]:
t:,0
s:1 2 3
:t,:1_s
s

0 2 3
1 2 3


Third: `"$ cd "~5#x;s,:0`. If the input matches `cd something`, put a 0 at the end of s.

And the last one, that ties things together: `~"0:"'*x;s+:.*" "\x`.

This one uses the `'` form of `bin`, which will return 0 if the first character of the line is a number, or 1 if it's a letter. It negates the result to only get `true` if the first character is a number.

In [124]:
"0:"'*'"121dsa"

0 0 0 1 1 1


In that case, we add to each element of s `s+:` the `eval` of the first word of the line `*" "\x`.

In [ ]:
s+:.*" "\x

Then, the solution for part1 is to `filter` the directories smaller than 10e5.

In [140]:
+/(1e5>)#t       /part1

1182909


For part 2, we should find the smallest directory that lets us free 40000000

In [161]:
&/((-4e7+*s)>)_t /part2

6999588


We use `filter-out`, which is the opposite of `filter`. Our filter predicate here is the total used size of our filesystem size (`*s`) minus of 4e7

In [170]:
*s /total used size (the size of the root dir)
-4e7 / 40000000 needed for the upgrade
-4e7+*s / space we need to recover

46592386
-40000000.0
6592386.0


In [172]:
((-4e7+*s)>)_t  /filter out the numbers that are less than this
((-4e7+*s)<)#t  /same, but using filter

8877344 9893050 17187447 22022022 6999588 7756031 46592386 18070237
8877344 9893050 17187447 22022022 6999588 7756031 46592386 18070237


We take the minimum with `&/`, giving us the final solution for part 2